In [1]:
#Question 1
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
    
businessCount = defaultdict(int)
userinformation = defaultdict(list)
userinformation_train = defaultdict(list)
userinformation_validation = defaultdict(list)
train_set =set()
validation_set =set()
all_set=set()
count = 0
train = open("train.txt", 'w')
validation =open("validation.txt", 'w')
for l in readGz("train.json.gz"):
    user,business = l['reviewerID'],l['itemID']
    userinformation[user].append(business)
    all_set.add(l['reviewerID'] + '-' + l['itemID'])
    if count < 100000:
        user,business = l['reviewerID'],l['itemID']
        userinformation_train[user].append(business)
        train_set.add(l['reviewerID'] + '-' + l['itemID'])
        count = count +1 
        train.write(user + '-' + business + ",1\n")
    else:
        user,business = l['reviewerID'],l['itemID']
        userinformation_validation[user].append(business)
        validation_set.add(l['reviewerID'] + '-' + l['itemID'])
        count = count +1 
        validation.write(user + '-' + business + ",1\n")


In [2]:
print(len(validation_set))
print(len(userinformation_validation))

100000
36402


In [3]:
import numpy as np
import random
user_sample = []
item_sample = []

for l in readGz("train.json.gz"):
    user_sample.append(l['reviewerID'])
    item_sample.append(l['itemID'])


In [4]:

random_user_sample =set()
random_item_sample =set()
for i in range(len(user_sample)):
    random_user_sample.add(user_sample[i])
    random_item_sample.add(item_sample[i])


In [5]:
random_user_sample =list(random_user_sample)  
random_item_sample =list(random_item_sample)  
random.shuffle(random_user_sample)
random.shuffle(random_item_sample)
count = 0
while count <100000 :
    pick_user = random.sample(random_user_sample,1)
    pick_item = random.sample(random_item_sample,1)
    if pick_item[0] not in userinformation_validation[pick_user[0]]:
        count += 1
        validation.write(pick_user[0] + '-' + pick_item[0] + ",0\n")
validation.close()
train.close()

In [6]:

businessCount = defaultdict(int)
totalPurchases = 0

for l in all_set:
    user,business = l.strip().split('-')
    businessCount[business] += 1
    totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases*0.515: 
        break


In [7]:
actul =[]
prediction =[]
accuracy = []
validation_fact = open('validation.txt').readlines()
for i in validation_fact:
    user,temporary = i.strip().split('-')
    item,fact = temporary.strip().split(',')
    actul.append(fact)
    if item in return1 :
        if int(fact) == 1 :
            accuracy.append(1)
        else :
            accuracy.append(0)
            prediction.append(1)
    else:
        if int(fact) == 0 :
            accuracy.append(1)
        else:
            accuracy.append(0)
        prediction.append(0)

In [8]:


accuracy_rate = sum(accuracy)/len(accuracy)
accuracy_rate

0.652115

In [9]:
#So the accuracy_rate is 0.630365.The prediction and actul situation can be seen from the list prediction and list


In [10]:
#question 3

In [9]:
cat_sample = []
for l in readGz("train.json.gz"):
    cat_sample.append(l['categories'])

IC_dict = dict(zip(item_sample,cat_sample))
UI_dict = userinformation


In [10]:
len(IC_dict)

19914

In [11]:
#New method to solve problem 3 and problem 4
dict2 = {}
list2 =set()
for item in IC_dict:
    for i in range(0,len(IC_dict[item])):
        for k in range (0,len(IC_dict[item][i])):
            if len(IC_dict[item][i][k]) > 0:
                    dict2[IC_dict[item][i][k]] = 0
for item in IC_dict:
    for i in range(0,len(IC_dict[item])):
        for k in range (0,len(IC_dict[item][i])):
            if len(IC_dict[item][i][k]) > 0:
                    dict2[IC_dict[item][i][k]] += 1            
    
len(dict2)

1042

In [12]:
dict4 = sorted(dict2.items(),key = lambda x:x[1],reverse = True)


In [13]:
dict4 = sorted(dict2.items(),key = lambda x:x[1],reverse = True)
list_high = []
list_medium = []
list_low = []
for i in range(20):
    list_high.append(dict4[i][0])
for i in range(20,50):
    list_medium.append(dict4[i][0])
for i in range(50,len(dict4)):
    list_low.append(dict4[i][0])

In [14]:
dict1 = {}
list1 =set()
for item in IC_dict:
    for i in range(0,len(IC_dict[item])):
        for k in range (0,len(IC_dict[item][i])):
            if len(IC_dict[item][i][k]) > 1:
                    list1.add(IC_dict[item][i][k]) 
    dict1[item] = list1
    list1 =set()

In [15]:
for i in validation_fact:
        user,temporary = i.strip().split('-')
        item,fact = temporary.strip().split(',')
        actul.append(fact)

In [111]:
    x=0
    count = 0
    accuracy= []
    prediction =[]
    cob_list = []
    count2 =0
    count2_list = []
    premium =0
    for i in validation_fact:
        user,temporary = i.strip().split('-')
        item,fact = temporary.strip().split(',')
        if item in return1:
            premium =3
        else:
            premium =0
        for j in range(0,len(UI_dict[user])):
            try:
                a = dict1[UI_dict[user][j]]
                b = dict1[item]
                cob = a&b
                cob_list = list(cob)
                for d in range(len(cob_list)):
                    if cob_list[d] in list_high:
                        count2 += 1
                    if cob_list[d] in list_medium:
                        count2 += 1
                    if cob_list[d] in list_low:
                        count2 += 1
                cob_list= []
                cob =[]
                count2_list.append(count2)
                count2 = 0
            except:
                continue
                count2_list.append(0)
        if sum(count2_list) != 0:
            count = max(count2_list)
            count2_list = []
        else:
            count = 0
            count2_list = []
        if count >= 3  :
            count = 0
            prediction.append(1)
        else:
            prediction.append(0)
            count = 0
    count = 0
    for i in range(0,len(prediction)):
        if prediction[i] == int(actul[i]):
            count = count +1
    accuracy_rate = count/len(prediction)
    print(x,accuracy_rate)
# Question 3(new method,better accuracy,0.63532)

0 0.945275


105371

In [ ]:
    test = open("pairs_Purchase.txt","r").readlines()
    test = test[1:]
    test = [i.strip() for i in test]
    prediction = []
    point = 0

    count = 0
    accuracy= []
    prediction =[]
    cob_list = []
    count2 =0
    count2_list = []
    premium = 0
    for i in test:
        user,item = i.strip().split('-')
        for j in range(0,len(UI_dict[user])):
            try:
                a = dict1[UI_dict[user][j]]
                b = dict1[item]
                cob = a&b
                cob_list = list(cob)
                for d in range(len(cob_list)):
                    if cob_list[d] in list_high:
                        count2 += 1
                    if cob_list[d] in list_medium:
                        count2 += 1
                    if cob_list[d] in list_low:
                        count2 += 1
                cob_list= []
                cob =[]
                count2_list.append(count2)
                count2 = 0
            except:
                continue
                count2_list.append(0)
        if sum(count2_list) != 0:
            count = max(count2_list)
            count2_list = []
        else:
            count = 0
            count2_list = []
        if count+premium > 3  :
            count = 0
            prediction.append(1)
        else:
            if item in return1:
                prediction.append(1)
                count = 0
            else:
                prediction.append(0)
                count = 0
sum(prediction)

In [108]:
count

100000

In [127]:
# Question 4(new method,better accuracy in test set,kaggle name Troy Peng accuracy : 0.63928
test = open("pairs_Purchase.txt","r").readlines()
test = test[1:]
test = [i.strip() for i in test]
prediction = []
point = 0

count = 0
accuracy= []
prediction =[]
cob_list = []
count2 =0
count2_list = []
premium = 0
for i in test:
    user,item = i.strip().split('-')
    for j in range(0,len(UI_dict[user])):
        try:
            a = dict1[UI_dict[user][j]]
            b = dict1[item]
            cob = a&b
            cob_list = list(cob)
            for d in range(len(cob_list)):
                if cob_list[d] in list_high:
                    count2 += 0.5
                if cob_list[d] in list_medium:
                    count2 += 1
                if cob_list[d] in list_low:
                    count2 += 1
            cob_list= []
            cob =[]
            count2_list.append(count2)
            count2 = 0
        except:
            count2_list.append(0)
            continue
    if sum(count2_list) != 0:
        count = max(count2_list)
        count2_list = []
    else:
        count = 0
        count2_list = []
    if count >  2.5 :
        count = 0
        prediction.append(1)
    elif item in return1:
        prediction.append(1)
        count = 0
    else:
        prediction.append(0)
        count=0
sum(prediction)

11326

In [17]:
import csv

with open('prediction_new.csv', 'w') as csv_file:
    csv_write = csv.writer(csv_file)
    for a in prediction:
        csv_write.writerow(str(a))



In [65]:
import csv

with open('prediction_new.csv', 'w') as csv_file:
    csv_write = csv.writer(csv_file)
    for a in prediction:
        csv_write.writerow(str(a))



In [ ]:
if 'Novelty' != IC_dict[item][i][k] and 'Clothing' != IC_dict[item][i][k] and 'Shoe' != IC_dict[item][i][k] and 'XOXO' != IC_dict[item][i][k] and 'Shoes' !=IC_dict[item][i][k] :
                    list1.add(IC_dict[item][i][k])
    dict1[item] = list1
    list1 =set()

In [ ]:
    def cosine_similarity(vector1, vector2):
        try:
            m = np.array(vector1).T
            n = np.array(vector2).T
            p= sum(m*n)
            k = (sum(m*m)**0.5)*(sum(n*n)**0.5)
            return p/k
        except:
            return 0
    from numpy import argmax
    whole_list = []
    for i in dict2:
        whole_list.append(i)
    count = 0
    accuracy= []
    prediction =[]
    cob_list = []
    count2 =0
    count2_list = []
    cosine = 0
    for i in validation_fact:
        user,temporary = i.strip().split('-')
        item,fact = temporary.strip().split(',')
        data1 = dict1[item]
        onehot_encoded_testitem=[]
        char_to_int_testitem = dict((cc, ii) for ii, cc in enumerate(whole_list))
        int_to_char_testitem = dict((ii, cc) for ii, cc in enumerate(whole_list))
        integer_encoded_testitem = [char_to_int_testitem[char] for char in data1]
        onehot_encoded_testitem = list()
        letter_testitem = [0 for _ in range(len(whole_list))]
        for value in integer_encoded_testitem:
            letter_testitem[value] = 1
        onehot_encoded_testitem.append(letter_testitem)    
        for j in range(0,len(UI_dict[user])):
            try:
                for k in range(0,len(IC_dict[UI_dict[user][j]])):
                    a = dict1[UI_dict[user][j]]
                judgement = 0 
            except:
                judgement = 1
                continue
        if judgement = 1 :
            prediction.append(0)
        else:
            cosine = cosine_similarity(onehot_encoded_testitem,)
    count = 0
    for i in range(0,len(prediction)):
        if prediction[i] == int(actul[i]):
            count = count +1
    accuracy_rate = count/len(prediction)
    print(x)
    print(accuracy_rate)
# Question 3(new method,better accuracy,0.63532)

In [ ]:

from numpy import array
def cosine_similarity(vector1, vector2):
    try:
        m = np.array(vector1).T
        n = np.array(vector2).T
        p= sum(m*n)
        k = (sum(m)**0.5)*(sum(n)**0.5)
        return p/k
    except:
        return 0
whole_list = []
for i in dict2:
    whole_list.append(i)

from numpy import argmax
# define input string
data = dict1["I402344648"]
print(data)
# define universe of possible input values
alphabet = whole_list
# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# integer encode input data
integer_encoded = [char_to_int[char] for char in data]
print(integer_encoded)
# one hot encode
onehot_encoded = list()
letter = [0 for _ in range(len(alphabet))]
for value in integer_encoded:
    letter[value] = 1
onehot_encoded.append(letter)       
data = dict1['I697650540']
alphabet = whole_list
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
integer_encoded = [char_to_int[char] for char in data]
onehot_encoded2 = list()
letter = [0 for _ in range(len(alphabet))]
for value in integer_encoded:
    letter[value] = 1
onehot_encoded2.append(letter)    
cosine_similarity(onehot_encoded2,onehot_encoded)